In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load and filter the data
csv_file_path = "log.csv"
df = pd.read_csv(csv_file_path)
df = df[df['Result'] == 'SUCCESS']

# Function to filter the data based on encoding, height, and cities
def filter_data(df, encoding, height, cities):
    if cities is not None:
        return df[(df['Encoding'] == encoding) & (df['Height'] == height) & (df['Cities'] == cities)]
    else:
        return df[(df['Encoding'] == encoding) & (df['Height'] == height) & (df['Cities'] != 2) & (df['Cities'] != 1000)]

# Function to compute the moving average
def moving_average(data, window_size):
    return data.rolling(window=window_size, min_periods=1).mean()

# Function to plot data
def plot_data(ax, df, encoding_label, window_size=3):
    smoothed_success = moving_average(df['Success'], window_size)
    ax.plot(df['Trains'], smoothed_success, label=encoding_label)

# Function to set up the plot
def setup_plot(ax):
    ax.set_xlim(left=0)
    ax.set_ylim(bottom=1)
    ax.set_yscale('log')
    ax.set_xlabel('Trains')
    ax.set_ylabel('Instances Solved')
    legend = ax.legend()
    for text in legend.get_texts():
        text.set_fontsize(text.get_fontsize() * 2)
    ax.grid(True)
    max_trains = ax.get_xlim()[1]
    ax.set_xticks(range(0, int(max_trains) + 1, 10))

# Function to save the plot
def save_plot(df, height, cities, filename, window_size=3):
    fig, ax = plt.subplots(figsize=(8, 6))
    for encoding, label in encodings:
        df_filtered = filter_data(df, encoding, height, cities)
        plot_data(ax, df_filtered, label, window_size)
    setup_plot(ax)
    plt.tight_layout(pad=0.1)
    plt.savefig(f"../../latex/documentation/images/{filename}", bbox_inches='tight')
    plt.show()
    plt.close(fig)

# Function to save the proportion plot
def save_proportion_plot(df, encoding, height, filename, window_size=3):
    df_filtered = df[(df["Encoding"] == encoding) & (df['Height'] == height) & (df['Cities'] == 2)]
    trains = df_filtered["Trains"]
    solving_proportion = df_filtered["Solving Proportion"]
    remaining_percentage = 1 - solving_proportion

    solving_proportion_smoothed = moving_average(solving_proportion, window_size)
    remaining_percentage_smoothed = moving_average(remaining_percentage, window_size)

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.fill_between(trains, solving_proportion_smoothed, color='lightcoral', alpha=0.6, label='Solving Proportion')
    ax.fill_between(trains, solving_proportion_smoothed, 1, color='lightblue', alpha=0.6, label='Remaining Percentage')
    ax.set_xlabel('Trains')
    ax.set_ylabel('Proportion')
    legend = ax.legend()
    for text in legend.get_texts():
        text.set_fontsize(text.get_fontsize() * 2)
    plt.tight_layout(pad=0.1)
    plt.savefig(f"../../latex/documentation/images/{filename}", format='eps', bbox_inches='tight')
    plt.show()
    plt.close()

# Function to save the horizon plot
def save_horizon_plot(df, filename, encoding, window_size=3):
    filtered_data = filter_data(df, encoding, 50, 2)
    smoothed_given_horizon = moving_average(filtered_data['Given Horizon'], window_size)
    smoothed_resulting_horizon = moving_average(filtered_data['Resulting Horizon'], window_size)
    
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_data['Trains'], smoothed_given_horizon, label='Given Horizon', marker='o')
    plt.plot(filtered_data['Trains'], smoothed_resulting_horizon, label='Resulting Horizon', marker='s')
    plt.xlabel('Number of Trains')
    plt.ylabel('Horizon')
    legend = plt.legend()
    for text in legend.get_texts():
        text.set_fontsize(text.get_fontsize() * 2)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(f"../../latex/documentation/images/{filename}", format='eps', bbox_inches='tight')
    plt.show()

In [ ]:

encodings = [
    ('encodings/multiple/grid/', 'Grid'),
    ('encodings/multiple/incremental/', 'Incremental'),
    ('encodings/multiple/weighted/', 'Weighted'),
    ('encodings/multiple/ordered_pipe/', 'Ordered'),
    ('encodings/multiple/path_assigning/', 'Path Assigning'),
    ('encodings/multiple/weighted_no_waiting/', 'Weighted*')
]

save_plot(df, 200, 2, 'sparse_s.eps')


In [ ]:
save_plot(df, 50, 2, 'normal_s.eps')

In [ ]:
save_plot(df, 50, None, 'dense_s.eps')

In [ ]:
save_proportion_plot(df, "encodings/multiple/weighted/", 100, "weighted_s.eps")

In [ ]:
save_proportion_plot(df, "encodings/multiple/weighted_no_waiting/", 100, "weighted_reduced_s.eps")

In [ ]:
save_horizon_plot(df, "horizons.eps", "encodings/multiple/ordered_pipe/")